# Coordinate systems and transforms of Synthesis AI face dataset

At first, we install some dependencies.

Installing `face_api_dataset` from the github it it's not already installed.

In [1]:
%%bash

(python -c "import face_api_dataset")
if [ $? -eq 0 ]
then
    echo "face_api_dataset is already installed"
else
    echo "installing face_api_dataset from github"
    pip install git+https://github.com/Synthesis-AI-Dev/face_api_dataset.git
fi

face_api_dataset is already installed


Installing `git lfs` from the github it it's not already installed.

In [2]:
%%bash

(git lfs install > /dev/null)
if [ $? -eq 0 ]
then
    echo "git lfs is already installed"
else
    echo "installing git lfs"
    case "$(uname -s)" in

       Darwin)
         brew install git-lfs
         ;;

       Linux)
         curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
         apt-get install git-lfs
         ;;

       *)
         echo "This cell can't install git lfs your OS. Please take care of it yourself."
         ;;
    esac
fi

git lfs is already installed


Pulling the test dataset for the example

In [3]:
%%bash

if [ -d "../test_dataset" ]
then
    echo "test_dataset is already present"
else
    echo "downloading test_dataset"
    git clone https://github.com/Synthesis-AI-Dev/face_api_dataset
    mv face_api_dataset/test_dataset ..
    rm -rf face_api_dataset
fi

test_dataset is already present


Finally we add some imports for the visualisation.

In [4]:
import cv2
%matplotlib inline

In [5]:
import numpy as np

We use `FaceApiDataset` class to access synthesis datasets.


In [6]:
from face_api_dataset import FaceApiDataset, Modality

**Warning!** Some of modalities requires additional libraries to be installed:
`SEGMENTS` and `RGB` modalities use `opencv-python` library,
 while `DEPTH`, `ALPHA` and `NORMALS` modalities
 use `tiffile`  and `imagecodecs` libraries for effective work with floating point tiff files.
 If dataset with these modalities will be created without corresponding libraries present, an `ImportError` is raised.


In [7]:
data_root = "../test_dataset"
dataset = FaceApiDataset(data_root,
                        modalities=[Modality.RGB, Modality.CAM_INTRINSICS, Modality.LANDMARKS_3D_IBUG68,
                                    Modality.LANDMARKS_IBUG68
                                   ])

In [8]:
len(dataset)

13

There are 13 items in the test dataset. Let's explore them closer.

In [9]:
item = dataset[0]
item2 = dataset[1]

Each item is a dict with different modalities as keys.

In [10]:
print(item.keys())

dict_keys([<Modality.RGB: 2>, <Modality.LANDMARKS_IBUG68: 7>, <Modality.LANDMARKS_3D_IBUG68: 13>, <Modality.CAM_INTRINSICS: 33>])


In this notebook we explore camera intrinsics and transistion between 3D and the screen coordinate systems.

In [20]:
intrinsics = item[Modality.CAM_INTRINSICS]
intrinsics

array([[1.55151515e+03, 0.00000000e+00, 2.56000000e+02],
       [0.00000000e+00, 1.55151515e+03, 2.56000000e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

The camera intrinsics are in OpenCV format:
$$\left [ \begin{matrix} f_x & 0 & c_x \\ 0 & f_y & c_y \\ 0 & 0 & 1 \end{matrix} \right ]$$,
where $f_x$ and $f_y$ are focal distances and $c_x$ and $c_y$ is an optical center
(2D coordinates of the point, camera is pointing at on the image).

Note, that units of distance in 3D are meters and units in 2D are pixels, the conversion coefficient is already included in $f_x$.

$c_x$ and $c_y$ are usually equal to the half of the image resolution as camera is normally pointing to the center of the image.

In [14]:
np.array(item[Modality.RGB].shape[:2]) / 2

array([256., 256.])

Now we can convert between camera and screen coordinate systems. Let's see how it is done, using ibug68 landmarks.

In [17]:
landmark_0 = item[Modality.LANDMARKS_3D_IBUG68]['0']
landmark_0

(-0.07423346489667892, 0.0001043081283569336, -1.0857646465301514)

At first, we need to check that landmark is in front of the camera (z coordinate is negative):

In [18]:
landmark_0[2] < 0


True

Now we can convert to screen coordinate system, using intrinsic matrix. However, our 3D camera coordinate system uses OpenGL axis (x right, y top, z towards the camera), and OpenCV system uses a different [one](https://docs.opencv.org/4.5.3/d9/d0c/group__calib3d.html) (x right, y bottom, z from the camera).
Thus before we apply the conversion, we need to convert between this coordinate systems, otherwise screen coordinates will be mirrored vertically.

In [22]:
def opengl_to_opencv(x):
    return np.array(x) * [1, -1, -1]

To compute landmark in screen coordinates we need to multiply 3D coordinates by the intrinsic matrix.

In [23]:
def cam_to_hom_screen(intrinsics, x):
    return intrinsics @ opengl_to_opencv(x)

In [24]:
cam_to_hom_screen(intrinsics, landmark_0)

array([162.78140398, 277.79391387,   1.08576465])

We get the result, but it is in homogeneous coordinates. We need to convert it to euclidian.

In [27]:
def from_homogeneous(x):
    x = np.array(x)
    return x[:2] / x[2]

In [28]:
def cam_to_screen(intrinsics, x):
    return from_homogeneous(cam_to_hom_screen(intrinsics, x))



In [29]:
cam_to_screen(intrinsics, landmark_0)

array([149.92328632, 255.85094777])

To check ourselves, let's compare with 2D location of this landmark we get from the info.json:

In [30]:
item[Modality.LANDMARKS_IBUG68]['0']



(149.9136, 255.8464)